In [19]:
import pandas as pd

# Load the CSV files
df60 = pd.read_csv("MSJO 60 01-03 2023.csv", delimiter=";")
df900 = pd.read_csv("MSJO 900 01-03 2023.csv", delimiter=";")

# Set PVALUE to float
df60["PVALUE"] = df60["PVALUE"].str.replace(",", ".").astype(float)
df900["PVALUE"] = df900["PVALUE"].str.replace(",", ".").astype(float)

# Duplicate deletion
df60.drop_duplicates(inplace=True)
df900.drop_duplicates(inplace=True)

# Delete rows with empty values
df60 = df60.dropna(subset=["MESS_ID", "VALUEDATE", "PVALUE"], how="any")
df900 = df900.dropna(subset=["MESS_ID", "VALUEDATE", "PVALUE"], how="any")

# Convert VALUEDATE to datetime
df60["VALUEDATE"] = pd.to_datetime(df60["VALUEDATE"])
df900["VALUEDATE"] = pd.to_datetime(df900["VALUEDATE"])

# Filter date range
df60 = df60[(df60['VALUEDATE'] > '2023-01-15') & (df60['VALUEDATE'] < '2023-03-20')]
df900 = df900[(df900['VALUEDATE'] > '2023-01-15') & (df900['VALUEDATE'] < '2023-03-20')]

# Define a function to filter outliers for each MESS_ID
def filter_outliers(df):
    # Group-wise calculation of quartiles and upper thresholds
    grouped = df.groupby("MESS_ID")
    filtered_dfs = []

    for name, group in grouped:
        # Calculate quartiles
        Q1 = group["PVALUE"].quantile(0.25)
        Q3 = group["PVALUE"].quantile(0.75)
        IQR = Q3 - Q1
        
        # Filter outliers
        filtered_group = group[(group["PVALUE"] >= Q1 - IQR) & (group["PVALUE"] <= Q3  + IQR)]
        
        # Check if there are filtered rows
        if not filtered_group.empty:
            # Append filtered group to list
            filtered_dfs.append(filtered_group)
    
    # Concatenate filtered dataframes if there are any
    if filtered_dfs:
        df_filtered = pd.concat(filtered_dfs)
        return df_filtered
    else:
        return None

# Filter outliers for df60 and df900 separately
df60_filtered = filter_outliers(df60)
df900_filtered = filter_outliers(df900)

if df60_filtered is not None:
    df60_filtered.sort_values(by=["MESS_ID", "VALUEDATE"], ascending=[True, True], inplace=True)

if df900_filtered is not None:
    df900_filtered.sort_values(by=["MESS_ID", "VALUEDATE"], ascending=[True, True], inplace=True)

# Save to CSV
if df60_filtered is not None:
    df60_filtered.to_csv("cleaned60.csv", index=False)

if df900_filtered is not None:
    df900_filtered.to_csv("cleaned900.csv", index=False)




In [20]:
# ARIMA Modell
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

df60 = pd.read_csv("cleaned60.csv")
df900 = pd.read_csv("cleaned900.csv")

# Convert 'VALUEDATE' column to datetime and set it as index
df60['VALUEDATE'] = pd.to_datetime(df60['VALUEDATE'])
df60.set_index('VALUEDATE', inplace=True)

df900['VALUEDATE'] = pd.to_datetime(df900['VALUEDATE'])
df900.set_index('VALUEDATE', inplace=True)

# Calculate the average value for each DataFrame
df60['avg_value'] = df60.mean(axis=1)
df900['avg_value'] = df900.mean(axis=1)

# Resample 60-second data to 15-minute intervals and handle missing values
df_60s_resampled = df60['avg_value'].resample('15T').mean()
df_60s_resampled = df_60s_resampled.fillna(df_60s_resampled.mean())

# Ensure df900's avg_value is handled similarly
df900['avg_value'] = df900['avg_value'].fillna(df900['avg_value'].mean())

# Train-Test Split
train_size_60s = int(len(df_60s_resampled) * 0.8)
train_size_15min = int(len(df900) * 0.8)

train_60s, test_60s = df_60s_resampled[:train_size_60s], df_60s_resampled[train_size_60s:]
train_15min, test_15min = df900['avg_value'][:train_size_15min], df900['avg_value'][train_size_15min:]

# Train ARIMA model on 60-second resampled data
arima_60s = ARIMA(train_60s, order=(5, 1, 0))
arima_60s_fit = arima_60s.fit()

# Train ARIMA model on 15-minute data
arima_15min = ARIMA(train_15min, order=(5, 1, 0))
arima_15min_fit = arima_15min.fit()

# Forecasting
arima_60s_forecast = arima_60s_fit.forecast(steps=len(test_60s))
arima_15min_forecast = arima_15min_fit.forecast(steps=len(test_15min))

# Evaluation
arima_60s_mse = mean_squared_error(test_60s, arima_60s_forecast)
arima_15min_mse = mean_squared_error(test_15min, arima_15min_forecast)

print(f'ARIMA 60-second resampled MSE: {arima_60s_mse}')
print(f'ARIMA 15-minute MSE: {arima_15min_mse}')


C:\Users\joaqu\AppData\Local\Temp\ipykernel_9072\1974744881.py:21: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_60s_resampled = df60['avg_value'].resample('15T').mean()
c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored 

ARIMA 60-second resampled MSE: 445.7801972791652
ARIMA 15-minute MSE: 310980.0456589194


In [21]:
# Prophet Modell
import pandas as pd
from prophet import Prophet

# Laden der 60-Sekunden-Daten
df60 = pd.read_csv('cleaned60.csv')
df60['VALUEDATE'] = pd.to_datetime(df60['VALUEDATE'])  # Umwandlung der 'VALUEDATE'-Spalte in Datetime
df60.set_index('VALUEDATE', inplace=True)  # Setzen der 'VALUEDATE'-Spalte als Index

# Laden der 15-Minuten-Daten
df900 = pd.read_csv('cleaned900.csv')
df900['VALUEDATE'] = pd.to_datetime(df900['VALUEDATE'])  # Umwandlung der 'VALUEDATE'-Spalte in Datetime
df900.set_index('VALUEDATE', inplace=True)  # Setzen der 'VALUEDATE'-Spalte als Index

# Fehlende Werte mit dem Mittelwert füllen
df60['avg_value'] = df60.mean(axis=1).fillna(df60.mean(axis=1).mean())
df900['avg_value'] = df900.mean(axis=1).fillna(df900.mean(axis=1).mean())

# Resample 60-Sekunden-Daten auf 15-Minuten-Intervalle
df_60s_resampled = df60['avg_value'].resample('15T').mean().fillna(df60.mean(axis=1).mean())

# Vorbereitung der Daten für Prophet
df_60s_prophet = df_60s_resampled.reset_index().rename(columns={'VALUEDATE': 'ds', 'avg_value': 'y'})
df_15min_prophet = df900.reset_index().rename(columns={'VALUEDATE': 'ds', 'avg_value': 'y'})

# Train-Test Split
train_size = int(len(df_60s_prophet) * 0.8)
train_60s_prophet = df_60s_prophet[:train_size]
test_60s_prophet = df_60s_prophet[train_size:]

train_15min_prophet = df_15min_prophet[:train_size]
test_15min_prophet = df_15min_prophet[train_size:]

# Training des Prophet-Modells auf 60-Sekunden-Daten
model_60s = Prophet()
model_60s.fit(train_60s_prophet)

# Training des Prophet-Modells auf 15-Minuten-Daten
model_15min = Prophet()
model_15min.fit(train_15min_prophet)

# Prognose
future_60s = model_60s.make_future_dataframe(periods=len(test_60s_prophet), freq='15T')
forecast_60s = model_60s.predict(future_60s)

future_15min = model_15min.make_future_dataframe(periods=len(test_15min_prophet), freq='15T')
forecast_15min = model_15min.predict(future_15min)

# Evaluation
from sklearn.metrics import mean_squared_error

test_60s_values = test_60s_prophet['y'].values
forecast_60s_values = forecast_60s.iloc[-len(test_60s_prophet):]['yhat'].values
mse_60s_prophet = mean_squared_error(test_60s_values, forecast_60s_values)

test_15min_values = test_15min_prophet['y'].values
forecast_15min_values = forecast_15min.iloc[-len(test_15min_prophet):]['yhat'].values
mse_15min_prophet = mean_squared_error(test_15min_values, forecast_15min_values)

print(f'Prophet 60-second resampled MSE: {mse_60s_prophet}')
print(f'Prophet 15-minute MSE: {mse_15min_prophet}')


c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
C:\Users\joaqu\AppData\Local\Temp\ipykernel_9072\982723542.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_60s_resampled = df60['avg_value'].resample('15T').mean().fillna(df60.mean(axis=1).mean())
17:13:43 - cmdstanpy - INFO - Chain [1] start processing
17:13:45 - cmdstanpy - INFO - Chain [1] done processing
17:13:46 - cmdstanpy - INFO - Chain [1] start processing
17:13:46 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'T' is deprecated and will be removed in a future version, pl

Prophet 60-second resampled MSE: 15909.63097637916
Prophet 15-minute MSE: 85273.33433007599


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def preprocess_data(data, is_energy_data=False):
    if is_energy_data:
        # If 'VALUEDATE' exists, preprocess it
        if 'VALUEDATE' in data.columns:
            # Convert 'VALUEDATE' to datetime
            data['VALUEDATE'] = pd.to_datetime(data['VALUEDATE'])
            # Extract year, month, day, hour, and minute
            data['Year'] = data['VALUEDATE'].dt.year
            data['Month'] = data['VALUEDATE'].dt.month
            data['Day'] = data['VALUEDATE'].dt.day
            data['Hour'] = data['VALUEDATE'].dt.hour
            data['Minute'] = data['VALUEDATE'].dt.minute
            # Drop the original 'VALUEDATE' column
            data.drop(columns=['VALUEDATE'], inplace=True)
    return data

def train_and_evaluate(energy_data_file, components_data_file, energy_id_col, components_id_col):
    # Load the energy data
    energy_data = pd.read_csv(energy_data_file)
    # Preprocess the energy data
    energy_data = preprocess_data(energy_data, is_energy_data=True)

    # Load the components data
    components_data = pd.read_csv(components_data_file, sep=';')

    # Merge energy data with components data on 'MESS_ID'
    merged_data = pd.merge(energy_data, components_data, left_on=energy_id_col, right_on=components_id_col)

    # Assuming you want to predict energy production based on the component ID and time features
    X = merged_data[['CNTL_ID', 'Year', 'Month', 'Day', 'Hour', 'Minute']]  # Independent variables
    y = merged_data['PVALUE']                                                # Dependent variable

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the testing set
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

# File paths for the two datasets
energy_data_minute = 'cleaned60.csv'
energy_data_15_minutes = 'cleaned900.csv'
components_data_file = 'TB Counter.csv'  # Considering ';' as separator

# Column names to use for merging
energy_id_col = 'MESS_ID'
components_id_col = 'CNTL_ID'

print("Results for energy produced every minute:")
train_and_evaluate(energy_data_minute, components_data_file, energy_id_col, components_id_col)

print("\nResults for energy produced every 15 minutes:")
train_and_evaluate(energy_data_15_minutes, components_data_file, energy_id_col, components_id_col)



Results for energy produced every minute:
Mean Squared Error: 394.5005984799122
Root Mean Squared Error (RMSE): 19.862039131970114
RMSE as a percentage of the mean: 377.01568757946455 %

Results for energy produced every 15 minutes:


c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean Squared Error: 57570.31463116803
Root Mean Squared Error (RMSE): 239.93814751132848
RMSE as a percentage of the mean: 808.1374311510068 %


c:\Users\joaqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
